In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from modules.loader import ImmothepDataLoader
loader = ImmothepDataLoader()
del(ImmothepDataLoader)

from modules.csvLoader import csvLoader

DATASET_PATH = '../datas/RAW/'

%load_ext autoreload
%autoreload 2

## Téléchargement des données

In [ ]:
loader.ensure_data_loaded()
del(loader)

## Lecture des CSV

In [ ]:
valeurs_foncieres = pd.read_csv(DATASET_PATH + "valeursfoncieres-2019.txt", sep='|', decimal=',', encoding='utf-8')

del(DATASET_PATH)

## Creation des jeux de données appartement et maison

In [ ]:
appartement = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Appartement']
maison = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Maison']

del(valeurs_foncieres)

cols = ['Surface reelle bati', 'Surface terrain', 'Valeur fonciere', 'Nombre pieces principales', 'Code postal']
maison = maison.dropna(how='any', subset=cols)
appartement = appartement.dropna(how='any', subset=cols)

del(cols)

# suppression des COLONNES vides 
maison = maison.dropna(axis=1, how='all')
appartement = appartement.dropna(axis=1, how='all')

csvLoader.save(appartement, 'appartement', save_index=False)
csvLoader.save(maison, 'maison', save_index=False)

del(appartement, maison)

## Définition de fonctions personnalisées

In [ ]:
def drop_extreme(df, colonne, low_qt, high_qt):
    limite_basse, limite_haute = df[colonne].quantile([low_qt, high_qt])
    return df.query("{low}<=`{colonne}`<={high}".format(low=limite_basse,high=limite_haute,colonne=colonne))

import math

def set_surface(row):
    if not math.isnan(row['Surface Carrez du 1er lot']):
        return row['Surface Carrez du 1er lot']
    return row['Surface reelle bati']

## Filtrage du jeu de données "maison"

In [ ]:
maison_filtre = csvLoader.load('maison')

maison_filtre = maison_filtre[maison_filtre['Nature mutation'] == 'Vente']
maison_filtre = maison_filtre.drop_duplicates(['Date mutation', 'Valeur fonciere', 'Code postal'], keep=False)
maison_filtre = drop_extreme(maison_filtre, 'Nombre pieces principales', 0.1, 0.9)

maison_filtre = maison_filtre[['Nature mutation', 'Valeur fonciere', 'Code postal', 'Voie', 'Commune', 'Surface Carrez du 1er lot', 'Nombre de lots', 'Type local', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain']]
maison_filtre['Surface'] = maison_filtre.apply(set_surface, axis=1)
maison_filtre = maison_filtre.drop(columns=['Surface Carrez du 1er lot', 'Surface reelle bati', 'Nombre de lots'])
maison_filtre = drop_extreme(maison_filtre, 'Surface', 0.1, 0.9)

maison_filtre = drop_extreme(maison_filtre, 'Surface terrain', 0.1, 0.90)
maison_filtre = drop_extreme(maison_filtre, 'Valeur fonciere', 0.05, 0.95)

# Prix au m²
maison_filtre['Prix metre carre'] = maison_filtre['Valeur fonciere']/maison_filtre['Surface']
valeur_surface = maison_filtre.groupby('Code postal')['Prix metre carre'].mean()

#Generation du csv prix au metre carre
csvLoader.save(valeur_surface, 'prix_metre_maison.csv')

maison_filtre = maison_filtre.merge(valeur_surface, on='Code postal')
maison_filtre = maison_filtre.drop(columns=['Prix metre carre_x'])
maison_filtre = maison_filtre.rename(columns={'Prix metre carre_y':'Prix metre carre'})
maison_filtre = maison_filtre.drop(columns=['Code postal'])
del(valeur_surface)

print(len(maison_filtre))
csvLoader.save(maison_filtre, 'maison_filtre', save_index=False)

## Filtrage du jeu de données "appartement"

In [ ]:
appartement_filtre = csvLoader.load('appartement')

appartement_filtre = appartement_filtre[appartement_filtre['Nature mutation'] == 'Vente']
appartement_filtre = appartement_filtre.drop_duplicates(['Date mutation', 'Valeur fonciere', 'Code postal'], keep=False)
appartement_filtre = drop_extreme(appartement_filtre, 'Nombre pieces principales', 0.1, 0.9)

appartement_filtre = appartement_filtre[appartement_filtre['Nombre de lots'] < 2]
appartement_filtre = appartement_filtre[['Date mutation', 'Nature mutation', 'Valeur fonciere', 'Code postal', 'Voie', 'Commune', 'Surface Carrez du 1er lot', 'Nombre de lots', 'Type local', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain']]
appartement_filtre['Surface'] = appartement_filtre.apply(set_surface, axis=1)
appartement_filtre = appartement_filtre.drop(columns=['Surface Carrez du 1er lot', 'Nombre de lots', 'Surface reelle bati'])

appartement_filtre = drop_extreme(appartement_filtre, 'Surface', 0.1, 0.9)
appartement_filtre = appartement_filtre[appartement_filtre['Surface terrain'].isna()]
appartement_filtre = appartement_filtre.drop(columns=['Surface terrain'])
appartement_filtre = drop_extreme(appartement_filtre, 'Valeur fonciere', 0.08, 0.92)

# Prix au m²
appartement_filtre['Prix metre carre'] = appartement_filtre['Valeur fonciere']/appartement_filtre['Surface']
valeur_surface = appartement_filtre.groupby('Code postal')['Prix metre carre'].mean()

# Generation du csv prix au metre carre
csvLoader.save(valeur_surface, 'prix_metre_appartement.csv')

appartement_filtre = appartement_filtre.merge(valeur_surface, on='Code postal')
appartement_filtre = appartement_filtre.drop(columns=['Prix metre carre_x'])
appartement_filtre = appartement_filtre.rename(columns={'Prix metre carre_y':'Prix metre carre'})
# Je test d'éliminer les extrèmes de prix au m²
appartement_filtre = drop_extreme(appartement_filtre, 'Prix metre carre', 0.08, 0.92)
appartement_filtre = appartement_filtre.drop(columns=['Code postal'])
del(valeur_surface)

print(len(appartement_filtre))
csvLoader.save(appartement_filtre, 'appartement_filtre', save_index=False)

del(appartement_filtre)

# Entrainement et sauvegarde de la regression linéaire

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

## Maison

In [ ]:
maison_filtre = csvLoader.load('maison_filtre')

maison_filtre = maison_filtre[['Valeur fonciere','Nombre pieces principales','Surface', 'Prix metre carre']]

X = pd.DataFrame(np.c_[maison_filtre['Nombre pieces principales'],maison_filtre['Surface'], maison_filtre['Prix metre carre']])
Y = maison_filtre['Valeur fonciere'] 
#base d'apprentissage et base de test 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
lmodellineaire = LinearRegression()
lmodellineaire.fit(X_train, Y_train)
pickle.dump(lmodellineaire, open("modele_lineaire_maison.p", "wb"))

del(lmodellineaire, X_train, Y_train, X_test, Y_test, X, Y, maison_filtre)

## Appartement

In [ ]:
appartement_filtre = csvLoader.load('appartement_filtre')

appartement_filtre = appartement_filtre[['Valeur fonciere','Nombre pieces principales','Surface', 'Prix metre carre']]

X = pd.DataFrame(np.c_[appartement_filtre['Nombre pieces principales'],appartement_filtre['Surface'], appartement_filtre['Prix metre carre']])
Y = appartement_filtre['Valeur fonciere']
#base d'apprentissage et base de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
lmodellineaire = LinearRegression()
lmodellineaire.fit(X_train, Y_train)
pickle.dump(lmodellineaire, open("modele_lineaire_appartement.p", "wb"))

del(lmodellineaire, X_train, Y_train, X_test, Y_test, X, Y, appartement_filtre)